# Path planning using GAN

## Loading packages

In [ ]:
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam
import tensorflow as tf
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

import os
import time
from pylab import *

## Loading data

In [ ]:
data_path = "D:\ML\PATHGAN\input\dataset"

### Prining a sample data

In [ ]:
sample_data = os.path.join(data_path, "00_010.csv")
sample_df = pd.read_csv(sample_data, index_col=None, header=None)
sample_df

### Visual representation

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(sample_df.values, interpolation='nearest')
plt.axis('off')
plt.show()

Load all the training paths

In [ ]:
def loadData(path=data_path):
    files = os.listdir(path)
    data = []
    labels = []
    for fn in files:
        if int(fn[0:2]) == 0:
            ffn = os.path.join(path, fn)
            df = pd.read_csv(ffn, index_col=None, header=None)
            df[df==2]=0
            data.append(df.values)
            label = int(fn[0:2])
            labels.append(label)
    data = np.array(data) 
    return data, labels

In [ ]:
data, labels = loadData()
print("Loaded: %s paths" % (len(labels)))

### Defining the GAN function

In [ ]:
class GAN():
    def __init__(self, img_rows=19, img_cols=13, channels=1):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
                                   optimizer=optimizer,
                                   metrics=['accuracy'])

        # Build and compile the generator
        self.generator = self.build_generator()
        self.generator.compile(loss='binary_crossentropy', optimizer=optimizer)

        # The generator takes noise as input and generated imgs
        z = Input(shape=(100,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator) takes
        # noise as input => generates images => determines validity 
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)

    def build_generator(self):

        noise_shape = (100,)

        model = Sequential()
        
        #Convolutional Layer1
        model.add(Dense(256, input_shape=noise_shape))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        #Layer2
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        #Layer3
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        #model.summary()

        noise = Input(shape=noise_shape)
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        img_shape = (self.img_rows, self.img_cols, self.channels)

        model = Sequential()
        
        #Conv. Layer1
        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        
        #Layer2
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        
        model.add(Dense(1, activation='sigmoid'))
        #model.summary()

        img = Input(shape=img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, X_train, y_, epochs=1000, batch_size=2):

        X_train = 2 * (X_train.astype(np.float32)) - 1
        X_train = np.expand_dims(X_train, axis=3)
        half_batch = int(batch_size / 2)

        for epoch in tqdm.trange(epochs, desc='Training'):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half batch of images
            idx = np.random.randint(0, X_train.shape[0], half_batch)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (half_batch, 100))

            # Generate a half batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, 100))

            # The generator wants the discriminator to label the generated samples as valid (ones)
            valid_y = np.array([1] * batch_size)

            # Train the generator
            g_loss = self.combined.train_on_batch(noise, valid_y)
            
        
    
    def save_pgan_model(self):
        self.generator.save_weights('D:\ML\PATHGAN\PathGANckpt.ckpt')
        self.generator.save('D:\ML\PATHGAN\PathGAN.h5')
        #self.generator.save('D:\ML\PATHGAN\PathGANFull')
        
        
    def load_pgan_model(self):
        self.generator = tf.keras.models.load_model('D:\ML\PATHGAN\PathGAN.h5')
        #self.generator.load_weights('D:\ML\PATHGAN\PathGANckpt.ckpt')
        #self.generator = tf.keras.models.load_model('D:\ML\PATHGAN\PathGANFull')
        print("Loaded the model from file")
        self.generator.summary()

### Build and Train the GAN

In [ ]:
gan = GAN()
start_time = time.time()
gan.train(data, labels, epochs=1000, batch_size=8)
#gan.save_pgan_model()
print("total training time: %s seconds" % (time.time() - start_time))

### Load from saved model

In [ ]:
#gan.load_pgan_model()

## Using the generator to generate a random path

In [ ]:
#gen_samples_row, gen_samples_col = 1,1
#count = gen_samples_row*gen_samples_col
count=1
noise = np.random.normal(0, 1, (count, 100))

# Generate images from noise data
gen_imgs = gan.generator.predict(noise)


#Rescale image pixels in 0 - 1
gen_imgs = 0.5 * gen_imgs + 0.5

# Display the generated path in matplotlib

%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(gen_imgs[0], cmap=plt.cm.gray)
plt.axis('off')
plt.show()